In [ ]:
import socket
import sys
import requests
import requests_oauthlib
import json

# SETTINGS
# Replace the values below with your own
# Twitter Consumer API keys
CONSUMER_KEY = "OEHl3FraeFTHNgdjOj8xT4hcWY"
CONSUMER_SECRET = "pitTZPaRJgyaAeqM6YVWCnLkqrsXkSd8wQEecKrZRzZaDHCQe4"
# Twitter Access token & access token secret
ACCESS_TOKEN = "1152234679065595906-JcMvsPncdnxSgcLrDiSa0OIdTGN8QP"
ACCESS_SECRET = "BnET56OLXTIV5ItlFyl3xe9NURxyzHTWPoTEcbEYVS5IR"
# You may customize the IP and Port, defaults should suffice though
TCP_IP = "localhost"
TCP_PORT = 39009

# Establish connection
my_auth = requests_oauthlib.OAuth1(
    CONSUMER_KEY, CONSUMER_SECRET, ACCESS_TOKEN, ACCESS_SECRET
)


def get_tweets():
    url = "https://stream.twitter.com/1.1/statuses/filter.json"
    query_data = [("language", "en"), ("locations", "-130,-20,100,50"), ("track", "#")]
    query_url = url + "?" + "&".join([str(t[0]) + "=" + str(t[1]) for t in query_data])
    response = requests.get(query_url, auth=my_auth, stream=True)
    print(query_url, response)
    return response


def send_tweets_to_spark(http_resp, tcp_connection):
    for line in http_resp.iter_lines():
        try:
            full_tweet = json.loads(line)
            tweet_text = full_tweet["text"]
            print("Tweet Text: " + tweet_text)
            print("------------------------------------------")
            tcp_connection.send(tweet_text + "\n")
        except:
            e = sys.exc_info()[0]
            print("Error: %s" % e)


if __name__ == "__main__":
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.bind((TCP_IP, TCP_PORT))
    s.listen(1)
    print("Waiting for TCP connection...")
    connection, address = s.accept()
    print("Connected... Starting getting tweets.")
    resp = get_tweets()
    send_tweets_to_spark(resp, connection)
